# Introdução às Redes Neurais com PyTorch

## Objetivo da Aula:

Nesta aula, vamos explorar passo a passo o funcionamento de uma rede neural artificial para **classificação binária** utilizando a biblioteca **PyTorch**. O código fornecido será usado como exemplo prático para entender cada etapa do processo de construção e treinamento de um modelo de aprendizado profundo (*deep learning*).

## Introdução ao Problema

O objetivo é criar um classificador binário que aprenda a prever se a soma de duas variáveis aleatórias é maior que 1.

> **Exemplo de regra:**  
> - Se $ x_1 + x_2 > 1 \Rightarrow y = 1 $ (classe positiva)  
> - Caso contrário, $ y = 0 $ (classe negativa)

Esse tipo de problema pode ser resolvido com modelos lineares simples, mas usaremos uma rede neural mais complexa para fins didáticos.

## Importando as bibliotecas

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Geração dos Dados Sintéticos

In [2]:
np.random.seed(42)
X_data = np.random.rand(200, 2)  # 200 amostras, 2 features
y_data = (X_data[:,0] + X_data[:,1] > 1.0).astype(int)

### Explicação:

 Ex.: y=1 se (x1 + x2 > 1.0), caso contrário y=0 (uma lógica simples).

- `np.random.rand(200, 2)` gera 200 pares de números entre 0 e 1.
- A linha `(X_data[:,0] + X_data[:,1] > 1.0).astype(int)` cria rótulos binários com base na regra definida.

📌 **Objetivo do seed**: Garantir reprodutibilidade dos resultados.

## Divisão em Treino e Teste

In [3]:
X_train, X_test, y_train, y_test = train_test_split(
    X_data, y_data, test_size=0.3, random_state=42
)

### Explicação:
- Usamos `train_test_split` do `sklearn.model_selection` para dividir os dados em:
  - 70% para **treino**
  - 30% para **teste**
- Isso evita que o modelo memorize os dados e garante que ele generalize bem.

## Conversão para Tensores do PyTorch

In [12]:
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32).view(-1,1)

X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.float32).view(-1,1)

### Explicação:
- O PyTorch trabalha com tensores (`torch.Tensor`) em vez de arrays NumPy.
- `.view(-1,1)` transforma o vetor de rótulos em uma coluna (formato exigido pela função de perda).

## Definição da Rede Neural

In [13]:
class AdvancedNet(nn.Module):
    def __init__(self, input_dim=2):
        super(AdvancedNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, 8)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(8, 4)
        self.fc3 = nn.Linear(4, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

### Estrutura da Rede:
| Camada | Função |
|--------|--------|
| `fc1`: Linear(2 → 8) | Primeira camada oculta |
| `ReLU` | Função de ativação não linear |
| `fc2`: Linear(8 → 4) | Segunda camada oculta |
| `ReLU` | Nova ativação |
| `fc3`: Linear(4 → 1) | Camada de saída (logits) |

💡 **Importante**: Não aplicamos `sigmoid` no final porque usaremos `BCEWithLogitsLoss`, que inclui isso internamente.

## Criando o Modelo

In [14]:
model = AdvancedNet(input_dim=2)

## Função de Perda e Otimizador

In [9]:

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

### Explicação:

- **`BCEWithLogitsLoss`**:
  - Combina `Sigmoid` + `Binary Cross Entropy Loss`.
  - Ideal para problemas de classificação binária.
- **`Adam`**:
  - Um otimizador adaptativo que ajusta automaticamente as taxas de aprendizado.
  - Mais eficiente que o SGD clássico.

## Loop de Treinamento

In [10]:
epochs = 20
for epoch in range(epochs):
    outputs = model(X_train_t)
    loss = criterion(outputs, y_train_t)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 5 == 0:
        print(f"Época {epoch+1}/{epochs}, Perda Treino: {loss.item():.4f}")

Época 5/20, Perda Treino: 0.6933
Época 10/20, Perda Treino: 0.6792
Época 15/20, Perda Treino: 0.6683
Época 20/20, Perda Treino: 0.6538


### Etapas do Treinamento:

1. **Forward pass**: Calcula a saída do modelo.
2. **Cálculo da perda**: Compara previsão com valor real.
3. **Backward pass (backpropagation)**: Calcula gradientes.
4. **Atualização dos pesos**: Usa o otimizador para ajustar os parâmetros.

📌 **Dica**: Imprimir a perda a cada poucas épocas ajuda a monitorar o progresso.

## Avaliação do Modelo

In [15]:
with torch.no_grad():
    logits_test = model(X_test_t)
    probs_test = torch.sigmoid(logits_test)
    preds_test = (probs_test > 0.5).float()

    acc = accuracy_score(y_test_t.numpy(), preds_test.numpy())
    print(f"\nAcurácia no teste: {acc*100:.2f}%")


Acurácia no teste: 50.00%


### Explicação:

- `torch.no_grad()` desativa o cálculo de gradientes (economiza memória).
- `torch.sigmoid` converte os *logits* para probabilidades entre 0 e 1.
- `preds_test = (probs_test > 0.5).float()` aplica o limiar de decisão.
- Usamos `accuracy_score` do `sklearn.metrics` para medir a acurácia.

## Considerações Finais

### Pontos-Chave Abordados:

- Como gerar dados sintéticos para aprendizado supervisionado.
- Como preparar os dados para uso no PyTorch.
- Estrutura básica de uma rede neural feedforward.
- Uso de funções de ativação (ReLU) e camadas densas (Linear).
- Uso de BCEWithLogitsLoss para classificação binária.
- Implementação do loop de treinamento com Adam.
- Avaliação de desempenho com acurácia.

### Sugestões para Próximos Passos:

- Adicionar regularização (Dropout, L2).
- Experimentar diferentes arquiteturas.
- Visualizar as fronteiras de decisão aprendidas.
- Trabalhar com conjuntos de dados reais (como Iris ou Breast Cancer do Scikit-learn).